# 2. Create an Incident from an Alert
Notebook to build all of the different stix objects associated with a Phishing Alert imitialising an Incident. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We take v3.0 of the Incident Core Extension specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are eight stages:

1. First setup the global parameters and retrieve the context memory
2. Create the objects from the phishing email Alert,
3. Add these objects to the ObservedData object
4. Then create the Indicator and get the user Identity object from context memory
5. Create the Sighting object with the SightingAlert extension
6. Create the Event and the Sequence objects
7. Create the Incident, with the Incident Core Extension, and add all of the objects to it
8. Save the Context Memory

Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



In [7]:
import sys
!{sys.executable} -m pip install stixorm
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


[notice] A new release of pip is available: 24.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [8]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [9]:
import json
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block,
)
from Utilities.local_make_general import invoke_create_incident_context, invoke_save_incident_context_block, invoke_get_from_company_block, invoke_get_from_user_block
from conv import conv

context_base = "../Orchestration/generated/os-triage/context_mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

## A.4. Create the Incident Object with empty objects

Create the Incident object and assign all of the objets to it

In [10]:
# 1. Setup path to form and results
inc_path ="SDO/Incident/phishing_incident.json"
results_path ="step1/incident_alert.json"
# 2. Setup the Sequence Object for the Event
#
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []
# 2. Invoke the Make Observed Data Block
incident_list = invoke_make_incident_block(inc_path, results_path, sequence_start_refs, sequence_refs, task_refs,event_refs, impact_refs, other_object_refs)
# 3. Add the record to the in-session bundles and lists
incident_results_context_path = results_base + "step1/incident_context.json"
result = invoke_create_incident_context(results_base + results_path, incident_results_context_path)
print(f" result->{result}")

{
    "type": "incident",
    "spec_version": "2.1",
    "id": "incident--557220b7-9758-4d48-84f6-e9a62b42e82f",
    "created": "2025-11-28T02:04:36.196Z",
    "modified": "2025-11-28T02:04:36.196Z",
    "name": "potential phishing",
    "extensions": {
        "extension-definition--ef765651-680c-498d-9894-99799f2fa126": {
            "extension_type": "property-extension",
            "investigation_status": "new",
            "incident_types": [
                "dissemination-phishing-emails"
            ]
        }
    }
}
🔍 DEBUG: Attempting to import module from: c:\projects\brett_blocks\Orchestration\generated\os-triage\common_files
🔍 DEBUG: Adding to sys.path: c:\projects\brett_blocks\Orchestration\generated\os-triage\common_files
🔍 DEBUG: Module imported successfully using direct import!
 result->{'context_result': ' incident context created -> incident--557220b7-9758-4d48-84f6-e9a62b42e82f\nstix_id -> incident--557220b7-9758-4d48-84f6-e9a62b42e82f'}


## B. Create the Objects to Document the Phishing Alert

### B.1 Create the Email Address for the Threat Actor

In [11]:
email_path = "SCO\EmailAddress\email_addr_THREAT.json"
results_path = "step1/email_addr_THREAT.json"
# 1. invoke the email object using method from ./Utilities/local_make_sco.py
threat_email_addr = invoke_make_email_addr_block(email_path, results_path)
# 2. add the record to the in-session bundles and incident lists
email_object_path = results_base + results_path
email_results_context_path = results_base + results_path + "context" 
result = invoke_save_incident_context_block(email_object_path, email_results_context_path)
print(f" result->{result}")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}
input data->{'stix_object': None}
from ports 
stix_object->None
context type->


TypeError: argument of type 'NoneType' is not iterable

### B.2 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

In [ ]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
to_email_addr = alert_data["to"]
TR_name = "Trusty Jones"
#
# 2. Setup variables and queries to get the objects from options
#
context_type = {
    "context_type": "users"
}
email_query = {
    "type" : "email-addr",
    "property": {
        "path": ["value"],
        "source_value": to_email_addr,
        "comparator": "EQ"
    }
}
reporter_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
reporter_email_addr = invoke_get_from_company_block(email_query, context_type, source_value=None, source_id=None)
usr_acct_query = {
    "type" : "user-account",
    "property": {
        "path": ["id"],
        "source_value": reporter_email_addr["belongs_to_ref"],
        "comparator": "EQ"
    }
}
reporter_account = invoke_get_from_company_block(usr_acct_query, context_type, source_value=None, source_id=None)
reporter_identity = invoke_get_from_company_block(reporter_ident_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "me"
}
TR_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
TR_identity = invoke_get_from_user_block(TR_ident_query, context_type, source_value=None, source_id=None)
#
print(f"\nreporter_email_addr-->{reporter_email_addr}")
print(f"\nreporter_account-->{reporter_account}")
print(f"\nreporter_identity-->{reporter_identity}")
print(f"\nTR_identity-->{TR_identity}")
#
# 3. save the objects to "unattached"
#
email_results_obj_path = results_base + "/step0/Usr1__email.json"
email_results_context_path = results_base + "/step1/context/usr2__email_context.json"
result = invoke_save_incident_context_block(email_results_obj_path, email_results_context_path)
print(f" result->{result}")
acct_results_obj_path = results_base + "/step0/Usr1__usr_acct.json"
acct_results_context_path = results_base + "/step1/context/usr2__usr_acct_context.json"
result = invoke_save_incident_context_block(acct_results_obj_path, acct_results_context_path)
print(f" result->{result}")
ident_results_obj_path = results_base + "/step0/Usr1__ident.json"
ident_results_context_path = results_base  + "/step1/context/usr2__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/TR_Identity__ident.json"
ident_results_context_path = results_base + "/step1/context/TR_Identity__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")

### B.3 Make the URL

In [ ]:
url_path = "SCO/URL/suspicious_url.json"
results_path = "step1/THREAT__url.json"
hyperlink = alert_data["url"]
# make the email object
sus_url = invoke_make_url_block(url_path, results_path, hyperlink)
# add the record to the in-session bundles and lists+
url_results_context_path = results_base + "/step1/THREAT__url_context.json"
sus_url_context = invoke_save_incident_context_block(results_base + results_path, url_results_context_path)
print(f" sus_url_context->{sus_url_context}")

### B.4 Make the Email Message

In [ ]:
msg_path = "SCO/EmailMessage/suspicious_email_msg.json"
results_path = "step1/SUSS__email_msg.json"
from_ref = threat_email_addr
to_ref = [reporter_email_addr]
cc_ref = []
bcc_ref = []
# make the email object
sus_msg = invoke_make_e_msg_block(msg_path, results_path, from_ref, to_ref, cc_ref, bcc_ref)
# add the record to the in-session bundles and lists
msg_results_context_path = results_base + "step1/THREAT__email_msg_context.json"
result = invoke_save_incident_context_block(results_base + results_path, msg_results_context_path)
print(f" result->{result}")

### B.4 Make the "derived-from" SRO

In [ ]:
# make the email object
relationship_type = "derived-from"
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_results_path = "step1/SRO_derived"
sus_relation = invoke_sro_block(sro_data_path, sro_results_path, sus_msg, sus_url, relationship_type)
sro_results_path2 = results_base + sro_results_path + "__rel.json"
# add the record to the in-session bundles and lists
msg_results_context_path = results_base + "step1/context/THREAT__email_SRO_context.json"
result = invoke_save_incident_context_block(sro_results_path2, msg_results_context_path)
print(f" result->{result}")

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [ ]:
# 1. Emulate "Add to List" Block
observed_list = []
observed_list.append(threat_email_addr)
observed_list.append(reporter_account)
observed_list.append(reporter_email_addr)
observed_list.append(sus_url)
observed_list.append(sus_msg)
observed_list.append(sus_relation)
# 2. Setup path to form and results
obs_path ="SDO/ObservedData/observation-alert.json"
results_path ="step1/observation-alert.json"
# 3. Invoke the Make Observed Data Block
obs_1 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_list)
# 4. Add the record to the in-session bundles and lists
obs_results_context_path = results_base + "step1/observation-alert_context.json"
result = invoke_save_incident_context_block(results_base + results_path, obs_results_context_path)
print(f" result->{result}")

## D. Create the Alert Sighting

Tbis then represents the sighting of potential phishing email as a group of elements

### D.1 Create a Indicator with a Pattern for the Phishing Email
but set its status to `unkown` to start with

In [ ]:
# 1. Setup pattern for indicator
pat1 = "[email-addr:value = '" + threat_email_addr["value"] + "' AND email:subject = '" + sus_msg["subject"] + "']"
# 2. Setup path to form and results
ind_path ="SDO/Indicator/indicator_alert.json"
results_path ="step1/indicator_alert.json"
# 3. Invoke the Make Observed Data Block
ind_1 = invoke_make_indicator_block(ind_path, results_path, pattern=pat1)
# 4. Add the record to the in-session bundles and lists
indicator_results_context_path = results_base + "step1/indicator_context.json"
result = invoke_save_incident_context_block(results_base + results_path, indicator_results_context_path)
print(f" result->{result}")

### D.2 Create the Sighting
Connecting the:
- observed-data object
- indicator object
- the reporter identity as the location

In [ ]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_alert.json"
results_path ="step1/sighting_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighted = ind_1
observation_list = [obs_1]
where_list = [reporter_identity]
# 2. Invoke the Make Observed Data Block
sight1 = invoke_sighting_block(sighting_data_path, results_path, observed=observation_list, sighted=sighted, where=where_list)
# 3. Add the record to the in-session bundles and lists
sighting_results_context_path = results_base + "step1/sighting_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path)
print(f" result->{result}")

### D.3 Promote Everything from Unattached to Other

Everything that uis connected in the Sighting should now be promoted, so move Unattached to the Other list

In [ ]:
# total_list = observed_list
# total_list.append(obs_1)
# total_list.append(ind_1)
# total_list.append(sight1)
# total_list.append(reporter_identity)

# obs_context_move_path = results_base + "step1/context/context_move.json"
# obs_context_move_results = results_base + "step1/context/context_move_results.json"
# result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

## E. Create the Event Objects
Tbis Event represents the receiving of the phishing email

In [ ]:
# 1. Setup path to form and results
event_data_path ="SDO/Event/event_alert.json"
results_path ="step1/event_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighting = sight1["id"]
# 2. Invoke the Make Observed Data Block
event_1 = invoke_make_event_block(event_data_path, results_path, changed_objects=None,sighting_refs=sighting)
# 3. Add the record to the in-session bundles and lists
event_results_context_path = results_base + "step1/context/event_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_results_context_path)
print(f" result->{result}")


### E.2 Then Create the Sequence Object for the Event

The Sequence object is created to place the event in a sequence, in this case a single-step

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_event_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "event"
sequenced_object = event_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
sequence_results_context_path = results_base + "step1/event_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sequence_results_context_path)
print(f" result->{result}")


### E.3 Since it is the First Sequence, Create the Start Sequence Object and Connect it

The Sequence object is created to place the event in a sequence, in this case a start-step

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_event_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "event"
on_completion = seq_E_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
event_results_context_path = results_base + "step1/event_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_results_context_path)
print(f" result->{result}")

## F. Create the Task Objects

Tbis Task represents the first Task to clarify the situation first,no real Impactyet

In [ ]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_alert.json"
results_path ="step1/task_alert.json"
# 2. Invoke the Make Observed Data Block
task_1 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
task_results_context_path = results_base + "step1/task_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_results_context_path)
print(f" result->{result}")

### F.2 Then Create the Sequence Object for the Task

The Sequence object is created to place the Task in a sequence, in this case a single-step

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
event_sequence_results_context_path = results_base + "step1/task_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_sequence_results_context_path)
print(f" result->{result}")


### F.3 Then Create the Sequence Start Object for the Tasks

The Sequence object is created to place the Task in a sequence, in this case a start-step

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_task_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "task"
on_completion = seq_T_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
task_seq_results_context_path = results_base + "step1/task_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_seq_results_context_path)
print(f" result->{result}")

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [ ]:
# Save the Tpe Refinery Context Memory File